In [2]:
pip install --upgrade pyod


[notice] A new release of pip available: 22.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install combo


[notice] A new release of pip available: 22.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
from pandas import json_normalize
import json
from datetime import datetime
import pytz
import matplotlib.pyplot as plt
import ruptures as rpt
from collections import Counter

In [5]:
columns = ['sensors.battery.temp',
       'sensors.battery.percent', 'sensors.battery.solarmv',
       'sensors.battery.current', 'sensors.battery.voltage',
       'sensors.battery.solarma', 'sensors.battery.charging',
        'solar_power', 'battery_power','power_difference']

In [6]:
# Reading back and setting the MultiIndex
labelled_data = pd.read_feather('MTDataBatch1\Labelled Data\Labelled Batch 1 - Updated.feather')
labelled_data.set_index(['DeviceId', 'time'], inplace=True)
labelled_data

Id  \
DeviceId        time                                   
015262001277846 2021-04-21 03:18:26+10:00      10046   
                2021-04-21 03:17:52+10:00      10049   
                2021-04-21 03:27:29+10:00      15189   
                2021-04-21 03:28:29+10:00      15190   
                2021-04-21 03:21:30+10:00      15191   
...                                              ...   
015262001276442 2021-11-22 13:01:23+11:00  195389569   
                2021-11-22 13:04:17+11:00  195391320   
                2021-11-23 13:05:02+11:00  196545125   
                2021-11-24 13:05:36+11:00  197957676   
                2021-11-24 13:05:02+11:00  197957683   

                                                                    CreatedAt  \
DeviceId        time                                                            
015262001277846 2021-04-21 03:18:26+10:00  2021-04-20 17:18:57.3601057 +00:00   
                2021-04-21 03:17:52+10:00  2021-04-20 17:18:57.9206698 +00:00   
                2021-04-21 03:27:29+10:00  2021-04-20 17:30:00.0973275 +00:00   
                2021-04-21 03:28:29+10:00  2021-04-20 17:30:00.5204188 +00:00   
                2021-04-21 03:21:30+10:00  2021-04-20 17:30:00.6604193 +00:00   
...                                                                       ...   
015262001276442 2021-11-22 13:01:23+11:00  2021-11-22 02:02:07.3194074 +00:00   
                2021-11-22 13:04:17+11:00  2021-11-22 02:04:57.8307232 +00:00   
                2021-11-23 13:05:02+11:00  2021-11-23 02:05:44.6179891 +00:00   
                2021-11-24 13:05:36+11:00  2021-11-24 02:06:18.7314233 +00:00   
                2021-11-24 13:05:02+11:00  2021-11-24 02:06:19.3460223 +00:00   

                                                                    UpdatedAt  \
DeviceId        time                                                            
015262001277846 2021-04-21 03:18:26+10:00  2021-04-20 17:18:57.4751133 +00:00   
                2021-04-21 03:17:52+10:00  2021-04-20 17:18:58.0006530 +00:00   
                2021-04-21 03:27:29+10:00  2021-04-20 17:30:00.1973672 +00:00   
                2021-04-21 03:28:29+10:00  2021-04-20 17:30:00.6554035 +00:00   
                2021-04-21 03:21:30+10:00  2021-04-20 17:30:00.7254268 +00:00   
...                                                                       ...   
015262001276442 2021-11-22 13:01:23+11:00  2021-11-22 02:02:07.4609948 +00:00   
                2021-11-22 13:04:17+11:00  2021-11-22 02:04:57.9257593 +00:00   
                2021-11-23 13:05:02+11:00  2021-11-23 02:05:44.7707739 +00:00   
                2021-11-24 13:05:36+11:00  2021-11-24 02:06:18.8826478 +00:00   
                2021-11-24 13:05:02+11:00  2021-11-24 02:06:19.4060168 +00:00   

                                          State             timeStamp  \
DeviceId        time                                                    
015262001277846 2021-04-21 03:18:26+10:00     1  2021-04-20T17:18:26Z   
                2021-04-21 03:17:52+10:00     1  2021-04-20T17:17:52Z   
                2021-04-21 03:27:29+10:00     1  2021-04-20T17:27:29Z   
                2021-04-21 03:28:29+10:00     1  2021-04-20T17:28:29Z   
                2021-04-21 03:21:30+10:00     1  2021-04-20T17:21:30Z   
...                                         ...                   ...   
015262001276442 2021-11-22 13:01:23+11:00     1  2021-11-22T02:01:23Z   
                2021-11-22 13:04:17+11:00     1  2021-11-22T02:04:17Z   
                2021-11-23 13:05:02+11:00     1  2021-11-23T02:05:02Z   
                2021-11-24 13:05:36+11:00     1  2021-11-24T02:05:36Z   
                2021-11-24 13:05:02+11:00     1  2021-11-24T02:05:02Z   

                                             event  sensors.battery.extpower  \
DeviceId        time                                                           
015262001277846 2021-04-21 03:18:26+10:00   moving                     False   
                2021-04

In [7]:
column_list = list(labelled_data.columns)
for column in column_list:
    print(column)
    print(labelled_data[column].isnull().values.any())
    print("-------------------")

Id
False
-------------------
CreatedAt
False
-------------------
UpdatedAt
False
-------------------
State
False
-------------------
timeStamp
False
-------------------
event
False
-------------------
sensors.battery.extpower
False
-------------------
sensors.battery.temp
False
-------------------
sensors.battery.percent
False
-------------------
sensors.battery.solarmv
False
-------------------
sensors.battery.current
False
-------------------
sensors.battery.voltage
False
-------------------
sensors.battery.solarma
False
-------------------
sensors.battery.charging
False
-------------------
location.bearing
False
-------------------
location.floor
False
-------------------
location.altitude
False
-------------------
location.longitude
False
-------------------
location.numSats
True
-------------------
location.source
False
-------------------
location.latitude
False
-------------------
location.speed
False
-------------------
location.accuracy
False
-------------------
sensors.accele

In [8]:
len(labelled_data.columns)

31

In [9]:
labelled_data.iloc[:,29].values

array([False, False, False, ..., False, False, False])

In [10]:
labelled_data.columns[6]

'sensors.battery.extpower'

In [11]:
# Training data : take all rows, and only device feature columns (From "sensors.battery.extpower" to "power_difference")
train_data = labelled_data.iloc[:,6:29]
# Drop columns that have string values, or even NaN values 
# Manual Inspection proved them as non important for malfunction detectio.
train_data = train_data.drop('location.source', axis=1)
train_data = train_data.drop('location.numSats', axis=1)
train_data = train_data.drop('sensors.accelerometer.y', axis=1)
train_data = train_data.drop('sensors.accelerometer.x', axis=1)
train_data = train_data.drop('sensors.accelerometer.z', axis=1)
# Labels is the array of values in "isDeviceWorking" -  convert to array of inetgers (0 or 1 values)
labels = labelled_data.iloc[:,29].values
test_data_label = labels.astype(int)

In [12]:
len(train_data.columns)

18

In [13]:
test_data_label

array([0, 0, 0, ..., 0, 0, 0])

In [14]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

normalized_test_data = pd.DataFrame(scaler.fit_transform(train_data))
normalized_test_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.0,0.375000,0.678696,0.000000,0.171937,0.631579,0.000000,0.0,0.498548,0.0,0.272651,0.920522,0.519553,0.059549,0.20245,0.000000,0.185898,0.541376
1,0.0,0.376582,0.677532,0.000000,0.172925,0.630031,0.000000,0.0,0.550609,0.0,0.254320,0.920596,0.519651,0.000000,0.14150,0.000000,0.186904,0.541376
2,0.0,0.387658,0.678696,0.000000,0.168972,0.631579,0.000000,0.0,0.736331,0.0,0.243320,0.920165,0.519078,0.021955,0.19400,0.000000,0.182912,0.541376
3,0.0,0.389241,0.678696,0.000000,0.176877,0.631579,0.000000,0.0,0.661008,0.0,0.247333,0.920036,0.519058,0.030376,0.26550,0.000000,0.190875,0.541376
4,0.0,0.378165,0.678696,0.000000,0.166008,0.631579,0.000000,0.0,0.557798,0.0,0.264178,0.920635,0.519076,0.052030,0.85355,0.000000,0.179926,0.541376
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1684078,0.0,0.541139,0.096624,0.819928,0.161067,0.089783,0.044481,0.0,0.000000,0.0,0.406648,0.399585,0.398616,0.000301,0.99220,0.040037,0.179616,0.593869
1684079,0.0,0.555380,0.095460,0.836735,0.168972,0.088235,0.055189,0.0,0.000000,0.0,0.406220,0.399569,0.398673,0.000000,0.12995,0.050694,0.186858,0.594189
1684080,0.0,0.643987,0.098952,0.804722,0.167984,0.091331,0.044481,0.0,0.000000,0.0,0.406220,0.399569,0.398673,0.000000,0.12995,0.039294,0.185932,0.589075
1684081,0.0,0.517405,0.088475,0.537015,0.163043,0.082043,0.034596,0.0,0.000000,0.0,0.400158,0.400058,0.400122,0.000902,0.15285,0.020395,0.181488,0.582629


In [15]:
# import numpy as np
# from sklearn.ensemble import HistGradientBoostingClassifier

# hGBC = HistGradientBoostingClassifier()
# #pca_train_data = pca.fit_transform(normalized_train_data)
# pca_test_data = hGBC.fit(normalized_test_data)

# print(pca.explained_variance_ratio_)
# print(sum(pca.explained_variance_ratio_))
# print(pca.singular_values_)

In [16]:
from __future__ import division
from __future__ import print_function

import os
import sys
from time import time

# supress warnings for clean output
import warnings

warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.ocsvm import OCSVM
from pyod.models.pca import PCA

from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn import metrics

from pyod.models.combination import aom, moa, average, maximization, majority_vote
from pyod.utils.data import generate_data
from pyod.utils.data import evaluate_print

In [17]:
def pred_labels(y_test, test_scores):
    fpr, tpr, thresholds = roc_curve(y_test, test_scores)
    cutoff = thresholds[np.argmax(tpr - fpr)]
    pred_label = []
    for each in test_scores:
        if each > cutoff:
            pred_label.append(1)
        else:
            pred_label.append(0)    
    return pred_label

In [18]:
def cal_eval(label, pred_label, scores):
    pr = round(metrics.precision_score(label, pred_label, average='macro'), ndigits=4)
    re = round(metrics.recall_score(label, pred_label, average='macro'), ndigits=4)
    f1 = round(metrics.f1_score(label, pred_label, average='macro'), ndigits=4)
    roc = round(roc_auc_score(label, scores), ndigits=4)
    return pr,re,f1,roc

In [19]:
X_train = normalized_test_data
X_test = normalized_test_data
y_train = test_data_label
y_test = test_data_label

In [20]:
outliers_fraction = 1 - (np.count_nonzero(y_train) / len(y_train))
outliers_percentage = round(outliers_fraction * 100, ndigits=4)
    
random_state = np.random.RandomState(42)
classifiers = {
        'Isolation Forest': IForest(contamination=outliers_fraction,
                                   random_state=random_state)
    ,
        'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
        'Local Outlier Factor (LOF)': LOF(
           contamination=outliers_fraction)
#     ,
#          'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction)
    }

n_clf = len(classifiers)
train_scores = np.zeros([X_train.shape[0], n_clf])
test_scores = np.zeros([X_test.shape[0], n_clf])
p_labels = np.zeros([X_test.shape[0], n_clf])

i = 0
train_duration = 0
test_duration = 0
for clf_name, clf in classifiers.items():
    t0 = time()
    clf.fit(X_train)
    t1 = time()
    test_sccore = clf.decision_function(X_test)
    t_t = time()
    test_mean_score = np.nanmean(test_scores)
    test_scores[np.isnan(test_scores)] = test_mean_score
    test_scores[:, i] = test_sccore
    p_labels[:, i] = pred_labels(y_test, test_sccore)
    i += 1
    print('Base detector %i is fitted for prediction' % i)
    train_duration += round(t1 - t0, ndigits=4)
    test_duration += round(t_t - t1, ndigits=4)
    
# standardize test score
test_scores_norm = standardizer(test_scores)
print(train_duration)

Base detector 1 is fitted for prediction
Base detector 2 is fitted for prediction
Base detector 3 is fitted for prediction
9279.706300000002


In [21]:
classifiers.items()

dict_items([('Isolation Forest', IForest(behaviour='old', bootstrap=False, contamination=0.2501367212898652,
    max_features=1.0, max_samples='auto', n_estimators=100, n_jobs=1,
    random_state=RandomState(MT19937) at 0x243C75E6E40, verbose=0)), ('K Nearest Neighbors (KNN)', KNN(algorithm='auto', contamination=0.2501367212898652, leaf_size=30,
  method='largest', metric='minkowski', metric_params=None, n_jobs=1,
  n_neighbors=5, p=2, radius=1.0)), ('Local Outlier Factor (LOF)', LOF(algorithm='auto', contamination=0.2501367212898652, leaf_size=30,
  metric='minkowski', metric_params=None, n_jobs=1, n_neighbors=20,
  novelty=True, p=2))])

In [22]:
# Build the Multi Layer Perceptron (MLP) model

import numpy as np
from tensorflow.keras.utils import to_categorical
from keras import models
from keras import layers
from keras.datasets import imdb
from time import time

In [23]:
train_x, test_x, train_y, test_y = train_test_split(test_scores_norm[:,0], y_test, test_size=0.9, random_state=random_state)

In [24]:
test_scores_norm.shape

(1684083, 3)

In [25]:
train_x.shape

(168408,)

In [26]:
def ensemble_mlp():
    df_columns = ['Precision', 'Recall', 'F1 score', 'AUC', 'Train time', 'Test time']
    enm_mlp_df = pd.DataFrame(columns=df_columns)

    t2 = time()
    model = models.Sequential()
    # Input - Layer

    model.add(layers.Dense(4, activation = "relu", input_dim=1))
    model.add(layers.Dense(20, activation = "relu"))
    model.add(layers.Dense(20, activation = "relu"))
    # Output- Layer
    model.add(layers.Dense(1, activation = "sigmoid"))
    model.summary()
    # compiling the model
    model.compile(
        optimizer = "adam",
        loss = "binary_crossentropy",
        metrics = ["accuracy"]
    )
    results = model.fit(
     train_x, train_y,
     epochs= 100,
     batch_size = 20,
     validation_data = (train_x, train_y)
    )
    t3 = time()

    pred_test = model.predict(test_scores_norm[:,0].reshape(len(test_scores_norm),1)).reshape(-1,1)

    t4 = time()
    train_time = t3 - t2
    test_time = t4 - t3
    train_time_mlp =  round(train_duration+train_time, ndigits=4)
    test_time_mlp = round(train_duration+test_time, ndigits=4)

    pred_nn = []
    for each in pred_test:
        if each[0] > 0.5:
            pred_nn.append(1)
        else:
            pred_nn.append(0)
    print(y_test.shape)
    print(pred_test.shape)
    pr_mlp, re_mlp, f1_mlp, roc_mlp = cal_eval(y_test, pred_nn, pred_test)
    enm_mlp = pd.DataFrame([pr_mlp, re_mlp, f1_mlp, roc_mlp, train_time_mlp, test_time_mlp]).transpose()
    enm_mlp.columns = df_columns
    enm_mlp_df = pd.concat([enm_mlp_df, enm_mlp], axis=0)
    return enm_mlp_df

In [27]:
df_columns = ['Precision', 'Recall', 'F1 score', 'AUC', 'Train time', 'Test time']
t_enm_mlp_df = pd.DataFrame(columns=df_columns)
for i in range(10):
    print(i)
    enm_mlp = ensemble_mlp()
    t_enm_mlp_df = pd.concat([t_enm_mlp_df, enm_mlp], axis=0, ignore_index=True)

0
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 8         
                                                                 
 dense_1 (Dense)             (None, 20)                100       
                                                                 
 dense_2 (Dense)             (None, 20)                420       
                                                                 
 dense_3 (Dense)             (None, 1)                 21        
                                                                 
Total params: 549
Trainable params: 549
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
8421/8421 [==============================] - 15s 2ms/step - loss: 0.5472 - accuracy: 0.7512 - val_loss: 0.5465 - val_accuracy: 0.7515
Epoch 2/100
8421/8421 [===================

8421/8421 [==============================] - 15s 2ms/step - loss: 0.5450 - accuracy: 0.7515 - val_loss: 0.5452 - val_accuracy: 0.7515
Epoch 51/100
8421/8421 [==============================] - 15s 2ms/step - loss: 0.5450 - accuracy: 0.7515 - val_loss: 0.5447 - val_accuracy: 0.7515
Epoch 52/100
8421/8421 [==============================] - 15s 2ms/step - loss: 0.5450 - accuracy: 0.7515 - val_loss: 0.5450 - val_accuracy: 0.7515
Epoch 53/100
8421/8421 [==============================] - 15s 2ms/step - loss: 0.5450 - accuracy: 0.7515 - val_loss: 0.5450 - val_accuracy: 0.7515
Epoch 54/100
8421/8421 [==============================] - 15s 2ms/step - loss: 0.5450 - accuracy: 0.7515 - val_loss: 0.5447 - val_accuracy: 0.7515
Epoch 55/100
8421/8421 [==============================] - 15s 2ms/step - loss: 0.5450 - accuracy: 0.7515 - val_loss: 0.5455 - val_accuracy: 0.7515
Epoch 56/100
8421/8421 [==============================] - 15s 2ms/step - loss: 0.5450 - accuracy: 0.7515 - val_loss: 0.5448 - val_a

                                                                 
Total params: 549
Trainable params: 549
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
8421/8421 [==============================] - 17s 2ms/step - loss: 0.5472 - accuracy: 0.7515 - val_loss: 0.5454 - val_accuracy: 0.7515
Epoch 2/100
8421/8421 [==============================] - 16s 2ms/step - loss: 0.5455 - accuracy: 0.7515 - val_loss: 0.5457 - val_accuracy: 0.7515
Epoch 3/100
8421/8421 [==============================] - 16s 2ms/step - loss: 0.5454 - accuracy: 0.7515 - val_loss: 0.5450 - val_accuracy: 0.7515
Epoch 4/100
8421/8421 [==============================] - 16s 2ms/step - loss: 0.5454 - accuracy: 0.7515 - val_loss: 0.5452 - val_accuracy: 0.7515
Epoch 5/100
8421/8421 [==============================] - 16s 2ms/step - loss: 0.5453 - accuracy: 0.7515 - val_loss: 0.5451 - val_accuracy: 0.7515
Epoch 6/100
8421/8421 [==============================] - 16s 2ms/step - lo

8421/8421 [==============================] - 16s 2ms/step - loss: 0.5451 - accuracy: 0.7515 - val_loss: 0.5449 - val_accuracy: 0.7515
Epoch 56/100
8421/8421 [==============================] - 16s 2ms/step - loss: 0.5450 - accuracy: 0.7515 - val_loss: 0.5448 - val_accuracy: 0.7515
Epoch 57/100
8421/8421 [==============================] - 16s 2ms/step - loss: 0.5450 - accuracy: 0.7515 - val_loss: 0.5452 - val_accuracy: 0.7515
Epoch 58/100
8421/8421 [==============================] - 16s 2ms/step - loss: 0.5451 - accuracy: 0.7515 - val_loss: 0.5449 - val_accuracy: 0.7515
Epoch 59/100
8421/8421 [==============================] - 16s 2ms/step - loss: 0.5450 - accuracy: 0.7515 - val_loss: 0.5449 - val_accuracy: 0.7515
Epoch 60/100
8421/8421 [==============================] - 16s 2ms/step - loss: 0.5450 - accuracy: 0.7515 - val_loss: 0.5449 - val_accuracy: 0.7515
Epoch 61/100
8421/8421 [==============================] - 16s 2ms/step - loss: 0.5451 - accuracy: 0.7515 - val_loss: 0.5448 - val_a

Epoch 4/100
8421/8421 [==============================] - 16s 2ms/step - loss: 0.5455 - accuracy: 0.7515 - val_loss: 0.5453 - val_accuracy: 0.7515
Epoch 5/100
8421/8421 [==============================] - 16s 2ms/step - loss: 0.5454 - accuracy: 0.7515 - val_loss: 0.5455 - val_accuracy: 0.7515
Epoch 6/100
8421/8421 [==============================] - 16s 2ms/step - loss: 0.5455 - accuracy: 0.7515 - val_loss: 0.5451 - val_accuracy: 0.7515
Epoch 7/100
8421/8421 [==============================] - 16s 2ms/step - loss: 0.5454 - accuracy: 0.7515 - val_loss: 0.5450 - val_accuracy: 0.7515
Epoch 8/100
8421/8421 [==============================] - 16s 2ms/step - loss: 0.5453 - accuracy: 0.7515 - val_loss: 0.5457 - val_accuracy: 0.7515
Epoch 9/100
8421/8421 [==============================] - 16s 2ms/step - loss: 0.5454 - accuracy: 0.7515 - val_loss: 0.5451 - val_accuracy: 0.7515
Epoch 10/100
8421/8421 [==============================] - 16s 2ms/step - loss: 0.5453 - accuracy: 0.7515 - val_loss: 0.5452 

...

Epoch 85/100
8421/8421 [==============================] - 17s 2ms/step - loss: 0.5496 - accuracy: 0.7515 - val_loss: 0.5496 - val_accuracy: 0.7515
Epoch 86/100
8421/8421 [==============================] - 17s 2ms/step - loss: 0.5496 - accuracy: 0.7515 - val_loss: 0.5495 - val_accuracy: 0.7515
Epoch 87/100
8421/8421 [==============================] - 17s 2ms/step - loss: 0.5496 - accuracy: 0.7515 - val_loss: 0.5501 - val_accuracy: 0.7515
Epoch 88/100
8421/8421 [==============================] - 18s 2ms/step - loss: 0.5496 - accuracy: 0.7515 - val_loss: 0.5495 - val_accuracy: 0.7515
Epoch 89/100
8421/8421 [==============================] - 17s 2ms/step - loss: 0.5496 - accuracy: 0.7515 - val_loss: 0.5497 - val_accuracy: 0.7515
Epoch 90/100
8421/8421 [==============================] - 17s 2ms/step - loss: 0.5496 - accuracy: 0.7515 - val_loss: 0.5496 - val_accuracy: 0.7515
Epoch 91/100
8421/8421 [==============================] - 17s 2ms/step - loss: 0.5496 - accuracy: 0.7515 - val_loss: 0

Epoch 34/100
8421/8421 [==============================] - 17s 2ms/step - loss: 0.5451 - accuracy: 0.7515 - val_loss: 0.5451 - val_accuracy: 0.7515
Epoch 35/100
8421/8421 [==============================] - 17s 2ms/step - loss: 0.5451 - accuracy: 0.7515 - val_loss: 0.5449 - val_accuracy: 0.7515
Epoch 36/100
8421/8421 [==============================] - 17s 2ms/step - loss: 0.5451 - accuracy: 0.7515 - val_loss: 0.5450 - val_accuracy: 0.7515
Epoch 37/100
8421/8421 [==============================] - 16s 2ms/step - loss: 0.5452 - accuracy: 0.7515 - val_loss: 0.5449 - val_accuracy: 0.7515
Epoch 38/100
8421/8421 [==============================] - 16s 2ms/step - loss: 0.5451 - accuracy: 0.7515 - val_loss: 0.5448 - val_accuracy: 0.7515
Epoch 39/100
8421/8421 [==============================] - 16s 2ms/step - loss: 0.5451 - accuracy: 0.7515 - val_loss: 0.5450 - val_accuracy: 0.7515
Epoch 40/100
8421/8421 [==============================] - 16s 2ms/step - loss: 0.5451 - accuracy: 0.7515 - val_loss: 0

Epoch 90/100
8421/8421 [==============================] - 20s 2ms/step - loss: 0.5451 - accuracy: 0.7515 - val_loss: 0.5449 - val_accuracy: 0.7515
Epoch 91/100
8421/8421 [==============================] - 20s 2ms/step - loss: 0.5451 - accuracy: 0.7515 - val_loss: 0.5450 - val_accuracy: 0.7515
Epoch 92/100
8421/8421 [==============================] - 20s 2ms/step - loss: 0.5450 - accuracy: 0.7515 - val_loss: 0.5453 - val_accuracy: 0.7515
Epoch 93/100
8421/8421 [==============================] - 20s 2ms/step - loss: 0.5451 - accuracy: 0.7515 - val_loss: 0.5449 - val_accuracy: 0.7515
Epoch 94/100
8421/8421 [==============================] - 20s 2ms/step - loss: 0.5449 - accuracy: 0.7515 - val_loss: 0.5450 - val_accuracy: 0.7515
Epoch 95/100
8421/8421 [==============================] - 20s 2ms/step - loss: 0.5450 - accuracy: 0.7515 - val_loss: 0.5451 - val_accuracy: 0.7515
Epoch 96/100
8421/8421 [==============================] - 20s 2ms/step - loss: 0.5450 - accuracy: 0.7515 - val_loss: 0

Epoch 39/100
8421/8421 [==============================] - 20s 2ms/step - loss: 0.5450 - accuracy: 0.7515 - val_loss: 0.5456 - val_accuracy: 0.7515
Epoch 40/100
8421/8421 [==============================] - 20s 2ms/step - loss: 0.5450 - accuracy: 0.7515 - val_loss: 0.5449 - val_accuracy: 0.7515
Epoch 41/100
8421/8421 [==============================] - 20s 2ms/step - loss: 0.5450 - accuracy: 0.7515 - val_loss: 0.5449 - val_accuracy: 0.7515
Epoch 42/100
8421/8421 [==============================] - 20s 2ms/step - loss: 0.5450 - accuracy: 0.7515 - val_loss: 0.5448 - val_accuracy: 0.7515
Epoch 43/100
8421/8421 [==============================] - 20s 2ms/step - loss: 0.5450 - accuracy: 0.7515 - val_loss: 0.5453 - val_accuracy: 0.7515
Epoch 44/100
8421/8421 [==============================] - 20s 2ms/step - loss: 0.5450 - accuracy: 0.7515 - val_loss: 0.5450 - val_accuracy: 0.7515
Epoch 45/100
8421/8421 [==============================] - 20s 2ms/step - loss: 0.5450 - accuracy: 0.7515 - val_loss: 0

Epoch 95/100
8421/8421 [==============================] - 20s 2ms/step - loss: 0.5450 - accuracy: 0.7515 - val_loss: 0.5448 - val_accuracy: 0.7515
Epoch 96/100
8421/8421 [==============================] - 20s 2ms/step - loss: 0.5450 - accuracy: 0.7515 - val_loss: 0.5451 - val_accuracy: 0.7515
Epoch 97/100
8421/8421 [==============================] - 20s 2ms/step - loss: 0.5450 - accuracy: 0.7515 - val_loss: 0.5448 - val_accuracy: 0.7515
Epoch 98/100
8421/8421 [==============================] - 20s 2ms/step - loss: 0.5450 - accuracy: 0.7515 - val_loss: 0.5448 - val_accuracy: 0.7515
Epoch 99/100
8421/8421 [==============================] - 21s 2ms/step - loss: 0.5450 - accuracy: 0.7515 - val_loss: 0.5450 - val_accuracy: 0.7515
Epoch 100/100
52628/52628 [==============================] - 49s 921us/step
(1684083,)
(1684083, 1)


In [28]:
y_test

array([0, 0, 0, ..., 0, 0, 0])

In [29]:
unique, counts = np.unique(y_test, return_counts=True)
dict(zip(unique, counts))

{0: 421251, 1: 1262832}

In [30]:
print('Ensemble mlp')
metrics = ['Precision', 'Recall', 'F1 score', 'AUC', 'Train time', 'Test time']
mean_mlp = []
std_mlp = []
for each in metrics:
    mean_mlp.append(round(t_enm_mlp_df[each].mean(), ndigits=4))
    std_mlp.append(round(t_enm_mlp_df[each].std(), ndigits=4))
print(mean_mlp)
print(std_mlp)
t_enm_mlp_df

Ensemble mlp
[0.3749, 0.5, 0.4285, 0.6122, 13237.1359, 9335.0387]
[0.0, 0.0, 0.0, 0.0056, 7038.404, 6.5451]


,Precision,Recall,F1 score,AUC,Train time,Test time
0,0.3749,0.5,0.4285,0.6141,10752.9015,9331.8830
1,0.3749,0.5,0.4285,0.6140,33264.7058,9328.5291
2,0.3749,0.5,0.4285,0.6142,10911.6899,9330.9788
3,0.3749,0.5,0.4285,0.6139,10991.8095,9330.2219
4,0.3749,0.5,0.4285,0.6140,10991.3474,9330.5435
5,0.3749,0.5,0.4285,0.6138,11131.8862,9343.7102
6,0.3749,0.5,0.4285,0.6138,10987.7867,9334.6220
7,0.3749,0.5,0.4285,0.5963,10956.8244,9330.8557
8,0.3749,0.5,0.4285,0.6141,11088.0132,9343.9685
9,0.3749,0.5,0.4285,0.6140,11294.3941,9345.0745


In [29]:
devices = set(labelled_data.index.get_level_values('DeviceId'))

In [1]:
devices

NameError: name 'devices' is not defined